In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
%cd gdrive/'My Drive'/gan_results/

/content/gdrive/My Drive/gan_results


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# get the training datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)

# prepare data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           num_workers=num_workers)

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
dataiter.next()
images, labels = dataiter.next()
labels
images
images.numpy()
images = images.numpy()
images[0]
np.squeeze(images[0])
# # get one image from the batch
# img = np.squeeze(images[0])

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Discriminator(nn.Module):

    def __init__(self, input_size, hidden_dim, output_size):
        super(Discriminator, self).__init__()
        
        # define all layers
        self.fc1=nn.Linear(input_size,hidden_dim*4)
        self.fc2=nn.Linear(hidden_dim*4,hidden_dim*2)
        self.fc3=nn.Linear(hidden_dim*2,hidden_dim)
        self.fc4=nn.Linear(hidden_dim,output_size)
        
        self.droppout=nn.Dropout(0.3)
        
        
    def forward(self, x):
        # flatten image
        x=x.view(-1,28*28)
        
        # pass x through all layers
        x=self.fc1(x)
        x=F.leaky_relu(x,0.2)
        x=self.droppout(x)
        
        x=self.fc2(x)
        x=F.leaky_relu(x,0.2)
        x=self.droppout(x)
        
        x=self.fc3(x)
        x=F.leaky_relu(x,0.2)
        x=self.droppout(x)
        
        out=self.fc4(x)
        

        
        # apply leaky relu activation to all hidden layers

        return out

In [7]:
class Generator(nn.Module):

    def __init__(self, input_size, hidden_dim, output_size):
        super(Generator, self).__init__()
        
        # define all layers
        self.fc1=nn.Linear(input_size,hidden_dim)
        self.fc2=nn.Linear(hidden_dim,hidden_dim*2)
        self.fc3=nn.Linear(hidden_dim*2,hidden_dim*4)
        self.fc4=nn.Linear(hidden_dim*4,output_size)
        self.droppout=nn.Dropout(0.3)
        

    def forward(self, x):
        # pass x through all layers
        x=self.fc1(x)
        x=F.leaky_relu(x,0.2)
        x=self.droppout(x)
        x=self.fc2(x)
        x=F.leaky_relu(x,0.2)
        x=self.droppout(x)
        x=self.fc3(x)
        x=F.leaky_relu(x,0.2)
        x=self.droppout(x)
        x=self.fc4(x)
        x=F.tanh(x)
        
        # final layer should have tanh applied
        
        return x

In [5]:
# Discriminator hyperparams

# Size of input image to discriminator (28*28)
input_size = 784
# Size of discriminator output (real or fake)
d_output_size = 1
# Size of *last* hidden layer in the discriminator
d_hidden_size = 32

# Generator hyperparams

# Size of latent vector to give to generator
z_size = 200
# Size of discriminator output (generated image)
g_output_size = 784
# Size of *first* hidden layer in the generator
g_hidden_size = 32

In [13]:
from torchsummary import summary
# instantiate discriminator and generator
D = Discriminator(input_size, d_hidden_size, d_output_size)
G = Generator(z_size, g_hidden_size, g_output_size)

# check that they are as you expect
summary(D, (784, 128))
summary(G, (32, 200))
print(D)
print()
print(G)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]         100,480
           Dropout-2                  [-1, 128]               0
            Linear-3                   [-1, 64]           8,256
           Dropout-4                   [-1, 64]               0
            Linear-5                   [-1, 32]           2,080
           Dropout-6                   [-1, 32]               0
            Linear-7                    [-1, 1]              33
Total params: 110,849
Trainable params: 110,849
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 0.00
Params size (MB): 0.42
Estimated Total Size (MB): 0.81
----------------------------------------------------------------
----------------------------------------------------------------
        Layer (type)               Output

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [15]:

# Calculate losses
def real_loss(D_out, smooth=False):
    # compare logits to real labels
    # smooth labels if smooth=True
    size=D_out.size(0)
    if smooth:
        labels=torch.ones(size)*0.9
    else:
        labels=torch.ones(size)

    criterion=nn.BCEWithLogitsLoss()
    loss = criterion(D_out.squeeze(),labels)
    return loss

def fake_loss(D_out):
    # compare logits to fake labels
    size=D_out.size(0)
    labels=torch.zeros(size)
    criterion=nn.BCEWithLogitsLoss()
    loss = criterion(D_out.squeeze(),labels)
    return loss

In [16]:
import torch.optim as optim

# learning rate for optimizers
lr = 0.002

# Create optimizers for the discriminator and generator
d_optimizer = optim.Adam(D.parameters(),lr=lr)
g_optimizer = optim.Adam(G.parameters(),lr=lr)

In [ ]:
import pickle as pkl
# training hyperparams
num_epochs = 1000

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 400

# Get some fixed data for sampling. These are images that are held
# constant throughout training, and allow us to inspect the model's performance
sample_size=16
fixed_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
fixed_z = torch.from_numpy(fixed_z).float()

# train the network
D.train()
G.train()
for epoch in range(num_epochs):
    
    for batch_i, (real_images, _) in enumerate(train_loader):
                
        batch_size = real_images.size(0)
        
        ## Important rescaling step ## 
        real_images = real_images*2 - 1  # rescale input images from [0,1) to [-1, 1)
        
        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================
                
        # 1. Train with real images

        # Compute the discriminator losses on real images
        # use smoothed labels
        
        d_optimizer.zero_grad()
        D_real=D(real_images)
        
        d_realloss=real_loss(D_real,smooth=True)
        
        # 2. Train with fake images
        
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z = torch.from_numpy(z).float()
        fake_images = G(z)
        
        # Compute the discriminator losses on fake images
        D_fake=D(fake_images)
        d_fakeloss=fake_loss(D_fake)
        
        # add up real and fake losses and perform backprop
        d_loss = d_realloss+d_fakeloss
        d_loss.backward()
        d_optimizer.step()
        
        
        # =========================================
        #            TRAIN THE GENERATOR
        # =========================================
        g_optimizer.zero_grad()
        
        # 1. Train with fake images and flipped labels
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z = torch.from_numpy(z).float()
        G_fake=G(z)
        
        # Compute the discriminator losses on fake images 
        # using flipped labels!
        DG_fake=D(G_fake)
        # perform backprop
        
        G_fakeloss=real_loss(DG_fake)
        g_loss=G_fakeloss
        g_loss.backward()
        g_optimizer.step()
        
        
        
        
        

        # Print some loss stats
        if batch_i % print_every == 0:
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item()))

    
    ## AFTER EACH EPOCH##
    # append discriminator loss and generator loss
    losses.append((d_loss.item(), g_loss.item()))
    
    # generate and save sample, fake images
    G.eval() # eval mode for generating samples
    samples_z = G(fixed_z)
    samples.append(samples_z)
    G.train() # back to train mode


# Save training generator samples
with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)


In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator')
plt.plot(losses.T[1], label='Generator')
plt.title("Training Losses")
plt.legend()
import pickle as pkl
with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)

In [34]:
from torchvision.utils import save_image

def save_images_to_colab(img, i):
  save_image(img.reshape((28,28)), 'gan_'+i+ '.png')

In [ ]:
rows = 10 # split epochs into 10, so 100/10 = every 10 epochs
cols = 4
fig, axes = plt.subplots(figsize=(7,12), nrows=rows, ncols=cols, sharex=True, sharey=True)
count=0
print(len(samples))
for sample, ax_row in zip(samples[::int(len(samples)/rows)], axes):
    print(count)
    for img, ax in zip(sample[::int(len(sample)/cols)], ax_row):
        img = img.detach()
        count+=1
        ax.imshow(img.reshape((28,28)), cmap='Greys_r')
        #save_images_to_colab(img, str(count))
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)        
        